In [9]:
with open('etriaikey.txt') as f:
    ai_key = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'etriaikey.txt'

- 하나의 이미지 검출

In [ ]:
img_file = '사진/cat.jfif'
img_type = img_file.split('.'0[-1]
